<a href="https://colab.research.google.com/github/vanderbilt-data-science/ai_summer/blob/main/Week3_Day2_LangChain_DeepDive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all relevant libraries

!pip install -q langchain
!pip install -q openai
!pip install -q tiktoken
!pip install -q huggingface_hub > /dev/null
!pip install -q transformers > /dev/null
!pip install -q pypdf
!pip install -q datasets
!pip install -q chromadb
!pip install -q deeplake
!pip install -q wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.2/892.2 kB 17.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 12.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Set OpenAI API Key

import os
from getpass import getpass

openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


# Langchain Deep Dive

In this notebook, we'll take a deeper look at some of the LangChain modules introduced yesterday. 

## Let's first start with **Models**:

There are 3 kinds of models you will come across when working with LangChain:

1. LLMs

  Large Language Models (LLMs) are models that take a text string as input and return a text string as output.

2. Chat Models

  Chat models are usually backed by a language model, but the APIs used for these models are often more structued than a simple LLM. These models take a list of Chat Messages as input and return a Chat Message. 

3. Text Embedding Models

  These models take text as an input and return a list of floats.

As mentioned above, Chat Models are a sub-type of Language Models in LangChain. There is a subtle difference between these, but LangChain provides a unified interface across these models. While the underlying APIs are significantly different, we often want to use them interchangeably. 

Let's take a look at how we call these two model types:

In [ ]:
# Use the OpenAI() wrapper to call OpenAI's LLMs

from langchain.llms import OpenAI

llm = OpenAI()

# Use the ChatOpenAI() wrapper to call OpenAI's Chat Model

from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

In [ ]:
# Text to Text interface:

llm.predict("say hi!")

'\n\nHi there!'

In [ ]:
# Text to Text interface:

chat_model.predict("say hi!")

'Hello! How may I assist you today?'

In [ ]:
# Message to Message interface

from langchain.schema import HumanMessage

llm.predict_messages([HumanMessage(content="say hi!")])

AIMessage(content='\n\nHi there! How can I help you?', additional_kwargs={}, example=False)

In [ ]:
# Message to Message interface

chat_model.predict_messages([HumanMessage(content="say hi!")])

AIMessage(content='Hello there! How may I assist you?', additional_kwargs={}, example=False)

As you can see, both model types provide very similar responses, and can be used interchangeably (most of the times). Depending on how you want your inputs and outputs to be structured, you can choose to select a particular type of model. 

### Large Language Models (LLMs) in LangChain

In [ ]:
# Selecting a specific type of model:

llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)

The ***n*** parameter in the OpenAI() call specifies the number of chat completions to generate for each prompt, while the ***best_of*** parameter specifies the number of completions to return. The best completion is determined by the highest log probability according to the OpenAI API.

In [ ]:
# Most basic LLM functionalty:

llm("Tell me a joke")

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

Generate: More broadly, you can call an LLM with a list of inputs, getting back a more complete response than just the text. This complete response includes things like multiple top responses, as well as LLM provider specific information

In [ ]:
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"]*15)

In [ ]:
len(llm_result.generations)

30

In [ ]:
llm_result.generations[0]

[Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side!', generation_info={'finish_reason': 'stop', 'logprobs': None}),
 Generation(text='\n\nWhy did the chicken cross the road?\n\nTo get to the other side.', generation_info={'finish_reason': 'stop', 'logprobs': None})]

In [ ]:
llm_result.generations[-1]

[Generation(text='\n\nWhen I was younger\nI thought that love\nI was something like a fairytale\nI would find my prince and they would be my prince\nAnd I would be his woman\nAnd the love I felt now\nI did because of you was just a part of a larger love experience\nI would never be able to for anything else\nBut love is what I am\nI am love', generation_info={'finish_reason': 'stop', 'logprobs': None}),
 Generation(text="\n\nI cannot pick a love poem\n\nI must choose a love song\n\nSo I can tell you a story\n\nOf what I feel in my heart\n\nWhen I'm with you, my love\n\nI know that I'm complete\n\nAnd I don't need anyone else\n\nBut you, my love,\n\nYou're the only one\n\nThat knows what I need\n\nAnd you, my love,\n\nYou're the only one\n\nThat knows what I want\n\nWe'll follow each other\n\nTo the end\n\nAnd we'll find our way\n\nTo the place of our dreams\n\nAnd we'll be happy until the end\n\nWe need each other\n\nTo stay alive\n\nWe're together\n\nAnd we're perfect\n\nFor each othe

In [ ]:
# Specific to OpenAI models - llm_output

llm_result.llm_output

{'token_usage': {'completion_tokens': 4371,
  'total_tokens': 4491,
  'prompt_tokens': 120},
 'model_name': 'text-ada-001'}

#### Managing Usage:

When working with models (especially with OpenAI where you're charged for each call), it's important to know how many tokens a particular request uses. It is also useful to manage context length so you're aware of how much more context you can provide to the LLM. This can be done with the "get_num_tokens" function:

In [ ]:
llm.get_num_tokens("tell me a joke")

4

#### Using open-source models:

You can also call models hosted on the HuggingFace Hub. For this, you will need a HuggingFace Hub API Key which you can generate here: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
from langchain import HuggingFaceHub

repo_id = "google/flan-t5-xl"

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})

from langchain import PromptTemplate, LLMChain 
## Note: This model requires a prompt template and chain (covered in more detail in the following sections)
## Please see model specifications in order to successfully run 
## a model hosted on HuggingFace Hub

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who won the FIFA World Cup in the year 1994? "

print(llm_chain.run(question))

The FIFA World Cup is a football tournament that is played every 4 years. The year 1994 was the 44th FIFA World Cup. The final answer: Brazil.


HuggingFace models can also be run locally using the **HuggingFacePipeline** class. 

**NOTE: DO NOT RUN THIS CODE IF YOU ARE NOT USING COLAB PRO**

In [ ]:
#from langchain import HuggingFacePipeline

#llm = HuggingFacePipeline.from_model_id(model_id="bigscience/bloom-1b7", task="text-generation", model_kwargs={"temperature":0, "max_length":64})

In [ ]:
#template = """Question: {question}

#Answer: Let's think step by step."""

#prompt = PromptTemplate(template=template, input_variables=["question"])

#llm_chain = LLMChain(prompt=prompt, llm=llm)

#question = "What is electroencephalography?"

#print(llm_chain.run(question))

### ChatModels in LangChain:

Chat models are a variation on language models. While chat models use language models under the hood, the interface they expose is a bit different. Rather than expose a “text in, text out” API, they expose an interface where “chat messages” are the inputs and outputs.

NOTE: The code here is likely to change as LangChain develops the correct abstractions for the ChatModels APIs. Please refer to the LangChain documentation on this section if this code stops working. 

https://python.langchain.com/en/latest/modules/models/chat.html

In [ ]:
chat_model = ChatOpenAI(temperature = 0) 

The temperature parameter gives the chat model some creative flexibility.0 means that the model will be more precise and less creative. 

The main way ChatModels are different from LLMs is in the use of a "schema" or a type of message being sent to the model.

You can get chat completions by passing one or more messages to the chat model. The response will be a message. The types of messages currently supported in LangChain are **AIMessage**, **HumanMessage**, **SystemMessage**, and **ChatMessage** – ChatMessage takes in an arbitrary role parameter. Most of the time, you’ll just be dealing with HumanMessage, AIMessage, and SystemMessage

In [ ]:
# import chat model specific modules:

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# basic functionality:

chat_model([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

As you can see, we passed in the Human Message, and received an AI 
Message. 

With these message types, you can provide multiple inputs:

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I like programming.")
]
chat_model(messages)

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

You can also pass in batches of messages using the **generate** function. This returns an LLMResult object:

In [ ]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat_model.generate(batch_messages)
result

KeyboardInterrupt: ignored

In [ ]:
# As seen before, you can get the token usage from this output as follows:

result.llm_output

#### Prompt Templates and ChatModels

We'll take another look at prompt templates in the next section, but we can also use templating with ChatModels.

ChatModels use ChatPromptTemplate, SystemMessagePromptTemplate and HumanMessagePromptTemplate 

In [ ]:
#creating a system message template
template="You are a helpful assistant that translates {input_language} to {output_language}."

In [ ]:
#using the "from_template" functionality to create the SystemMessagePromptTemplate
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [ ]:
#creating a template for what the human message will look like:
human_template="{text}"

In [ ]:
#using the "from_template" functionality to create the HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
#combining the Human and System prompts in the ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

Now that we have the chat prompt template ready, we can use format_prompt to provide the inputs to the model. We can also use this to convert the input to a string or a Message object, depending on whether you are using a chatModel or llm. Here, we use the to_messages() functionality to convert the prompt to a message object for the chat model's input.

In [ ]:
chat_model(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

We can also provide this prompt to a chain (something we will see in the next few sections):

In [ ]:
chain = LLMChain(llm=chat_model, prompt=chat_prompt)

chain.run(input_language="English", output_language="French", text="I love programming.")

"J'adore la programmation."

#### Streaming

We often want chat models to act as if they are writing the response live (just like on the chatGPT web interface). We can do that through ChatOpenAI's callback handling. 

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
resp = chat([HumanMessage(content="Write me a song about sparkling water.")])

Verse 1:
Bubbles rising to the top
A refreshing drink that never stops
Clear and crisp, it's oh so pure
Sparkling water, I can't ignore

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Verse 2:
No sugar, no calories, just H2O
A drink that's good for me, don't you know
With lemon or lime, you're even better
Sparkling water, you're my forever

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Bridge:
You're my go-to drink, day or night
You make me feel so light
I'll never give you up, you're my true love
Sparkling water, you're sent from above

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Outro:
Sparkling water, you're the one for me
I'll 

### Text Embedding Models

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

You will most commonly come across the Embeddings provided by OpenAI:

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

text = "This is a test document."

You can use two embedding methods: embed_documents and embed_query. The largest difference is that these two methods have different interfaces: one works over multiple documents, while the other works over a single document.

In [ ]:
query_result = embeddings.embed_query(text)

In [ ]:
doc_result = embeddings.embed_documents([text])

## Prompts

The new way of programming models is through prompts. A “prompt” refers to the input to the model. This input is rarely hard coded, but rather is often constructed from multiple components. A PromptTemplate is responsible for the construction of this input. LangChain provides several classes and functions to make constructing and working with prompts easy.

There are four aspects of Prompts in LangChain:

1. LLM Prompt Templates: 
How to use PromptTemplates to prompt Language Models.
2. Chat Prompt Templates: 
How to use PromptTemplates to prompt Chat Models.
3. Example Selectors: 
Often times it is useful to include examples in prompts. These examples can be hardcoded, but it is often more powerful if they are dynamically selected. This section goes over example selection.
4. Output Parsers: 
Language models (and Chat Models) output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output Parsers are responsible for (1) instructing the model how output should be formatted, (2) parsing output into the desired formatting (including retrying if necessary).

### LLM Prompt Templates

On Day 1, we worked with a brief example on general LLM prompt templates. Below is code to review how we can create a prompt template.

In [ ]:
from langchain import PromptTemplate

template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt.format(product="colorful socks")

'\nI want you to act as a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n'

We can also set up our prompt template to take multiple inputs

In [ ]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="Tell me a {adjective} joke about {content}."
)
multiple_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

You can also create a PromptTemplate using the `from_template` method. Langchain will automatically infer the input_variables based on the template passed.

In [ ]:
template = "Tell me a {adjective} joke about {content}."

prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables

prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

You can save your prompt template to a file for easy reuse across applications. Langchain currently supports saving prompts to JSON and YAML files. 

In [ ]:
prompt_template.save("awesome_prompt.json") # Save to JSON file

In [ ]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt("awesome_prompt.json")  # Load our prompt from file

print(loaded_prompt)

loaded_prompt.format(adjective = "hilarious", content = "AI")

input_variables=['adjective', 'content'] output_parser=None partial_variables={} template='Tell me a {adjective} joke about {content}.' template_format='f-string' validate_template=True


'Tell me a hilarious joke about AI.'

### Chat Prompt Templates

Chat Models work a little differently from general LLM models, in that the chat models keep track of every message, and assign each as coming from either a Human, AI, or System. As the names imply, the Human messages are from the human sending prompts, the AI messages are the responses from the AI model, and the System messages are set by the Prompt Template and contain instructions for the model to follow. Let's look at an example. 

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

To create a message template associated with a role, you use MessagePromptTemplate. Below is an example using the SystemMessagePromptTemplate to set up system instructions for the AI to follow during the conversation.

In [ ]:
template="You are a helpful assistant that translates {input_language} to {output_language}."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)


Generally, the Human message will contain fully-variable text, unless you want to prefix every human message with a hard-coded message.

In [ ]:
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

After that, you can build a ChatPromptTemplate from one or more MessagePromptTemplates. You can use ChatPromptTemplate’s `format_prompt` – this returns a PromptValue, which you can convert to a string or Message object, depending on whether you want to use the formatted value as input to an llm (which needs a string) or chat model (which needs a Message object).



In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

### Example Selectors

Sometimes, giving a model an example prompt and answer helps align the model's output to be closer to what you want. We can use Langchain's FewShotPromptTemplate and ExampleSelector to do just that.

To do this, we first need to create a set of example questions and answers.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

Next, we need to create a formatter for these examples by using a PromptTemplate.

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Finallly, we feed this into Langchain's FewShotPromptTemplate.

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

What if we don't want to send all of the examples to our template, and instead use only the ones that are most relevant to the current input? We can use an ExampleSelector that allows the prompt template to choose the most relevant examples.

Let's start with the SemanticSimilarityExampleSelector, which selects few shot examples based on cosine similarity to the input using a vector store. 

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Then, we can give our ExampleSelector to the FewShotPromptTemplate.

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

There are several other ExampleSelectors that use different methods to choose the relevant examples. [Check them out](https://python.langchain.com/en/latest/modules/prompts/example_selectors.html), or create your own custom ExampleSelector.

### Output Parsers

Many times we want to be able to get structured information from the text that our language model outputs. For this, we can use output parsers, which are classes in Langchain that structure language model responses. 

There are two main methods an output parser must implement: 

- `get_format_instructions()`: returns a string containing instructions for how the output of a language model should be formatted
- `parse(str)`: takes in a string (response from the language model) and parses it into some structure.

Additionally, there is an optional method, called `parse_with_prompt(str)`, that takes in a string (response from a model) and the prompt that generated that response and parses it into some structure. This method is used when the OutputParser needs information from the prompt in order to parse the output.

Read about the different kinds of OutputParsers [here](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html). For now, we will start with a simple parser called StructuredOutputParser. If you want a more powerful and customizable parser, I recommend you look at the example code for PydanticOutputParser.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

First, define a response schema.

In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

Next, use the parser to get the instructions from your schema, and insert that into the PromptTemplate.

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

Now, you can format the prompt to the LLM, and parse the returned result

In [ ]:
model = OpenAI(temperature=0)

In [ ]:
_input = prompt.format_prompt(question="what's the capital of france")
output = model(_input.to_string())

In [ ]:
output_parser.parse(output)

You could also use this in a chat model.

In [ ]:
chat_model = ChatOpenAI(temperature=0)

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
_input = prompt.format_prompt(question="what's the capital of france")
output = chat_model(_input.to_messages())

In [ ]:
output_parser.parse(output.content)

### CommaSeparatedListOutputParser

This is another parser that formats the response into a comma separated list. 

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
output_parser = CommaSeparatedListOutputParser()

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
model = OpenAI(temperature=0)

In [ ]:
_input = prompt.format(subject="ice cream flavors")
output = model(_input)

In [ ]:
output_parser.parse(output)

### PydanticOutputParser

[Pydantic](https://pypi.org/project/pydantic/) is a Python library that allows you to implement data validation and settings management in Python. Langchain incorporates Pydantic into its PydanticOutputParser to allow for a powerful output formatting. 

In [ ]:
!pip install pydantic

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

Define your output structure as a class.

In [ ]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

Insert the class into the output parser.

In [ ]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

Construct the prompt template with the output parser instructions.

In [ ]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [ ]:
output = model(_input.to_string())

In [ ]:
parser.parse(output)

## Memory

By default, LLMs and ChatModels treat each query as independent and are stateless. However, in some applications (especially chatbots) it is necessary to remember previous interactions, both short and long term. 

The most basic memory module is the **ChatMessageHistory** class. While not used very frequently, this can be useful for saving messages outside of a chain. 

In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [ ]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

### ConversationBufferMemory - The simplest memory type in LangChain

ConversationBufferMemory is just a wrapper around ChatMessageHistory that extracts the messages in a variable.

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

We can also get the history as a list of message by passing the "return_messages=True" flag in ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}, example=False),
  AIMessage(content='whats up?', additional_kwargs={}, example=False)]}

### Using Memory in a Chain - Taking it one step further

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

#A New Chain! ConversationChain works well to incorporate memory and have a more "chatbot" like interaction
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [ ]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Tell me about yourself.
AI:

> Finished chain.


" Sure! I'm an AI created to help people with their everyday tasks. I'm programmed to understand natural language and provide helpful information. I'm also constantly learning and updating my knowledge base so I can provide more accurate and helpful answers."

So as you can see, the chain and model are able to keep track of the conversation in previous instances. 

### Saving Message History

You may want to export this chat history to a file to keep track of outputs. This can be done easily by first converting the messages to python dictionaries and then saving that to a json or txt file. 

In [ ]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

dicts = messages_to_dict(history.messages)

In [ ]:
dicts

[{'type': 'human',
  'data': {'content': 'hi!', 'additional_kwargs': {}, 'example': False}},
 {'type': 'ai',
  'data': {'content': 'whats up?', 'additional_kwargs': {}, 'example': False}}]

You can also load messages from the dictionary we created above

In [ ]:
new_messages = messages_from_dict(dicts)
new_messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

### Different Memory Types

LangChain provides a few different types of memory that are useful for different use cases. We just took a look at **ConversationBufferMemory.**

#### ConversationBufferWindowMemory

ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

from langchain.llms import OpenAI
from langchain.chains import ConversationChain


conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [ ]:
conversation_with_summary.predict(input="What's their issues?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:

> Finished chain.


" The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected."

In [ ]:
conversation_with_summary.predict(input="Is it going well?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:

> Finished chain.


" Yes, it's going well so far. We've already identified the problem and are now working on a solution."

In [ ]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="What's the solution?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:  Yes, it's going well so far. We've already identified the problem and are now working on a solution.
Human: What's the solution?
AI:

> Finished chain.


" The solution is to reset the router and reconfigure the settings. We're currently in the process of doing that."

As you can see, from a programmatic point of view, using this type of memory is very similar to using the basic ConversationBufferMemory. This goes for pretty much all memory types. Note that some memory types do require some additional inputs such as prompt templates. 

Let's take a look at another example:

#### Entity Memory

Entity memory allows the models to remember things about specific entities. It extracts information on entities (using LLMs) and builds up its knowledge about that entity over time (also using LLMs).


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [ ]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great project! What kind of project are they working on?'

In [ ]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [ ]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like an interesting project! What kind of memory structures are they trying to add?'

In [ ]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great idea! How will the key-value store work?'

In [ ]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Deven and Sam are working on a hackathon project together, attempting to add more complex memory structures to Langchain, including a key-value store for entities mentioned so far in the conversation.'

In [ ]:
#Examining the memory store following the conversation:

conversation.memory.entity_store.store

{'Deven': 'Deven and Sam are working on a hackathon project together, attempting to add more complex memory structures to Langchain, including a key-value store for entities mentioned so far in the conversation.',
 'Sam': 'Sam is working on a hackathon project with Deven, attempting to add more complex memory structures to Langchain, including a key-value store for entities mentioned so far in the conversation.',
 'Langchain': 'Langchain is a project that is trying to add more complex memory structures, including a key-value store for entities mentioned so far in the conversation.',
 'Key-Value Store': 'A key-value store that stores entities mentioned in the conversation.'}

As you can see, different memory types can be useful in different situations. These are relatively similar when it comes to implementing them, with minor additional parameters required depending on the type of memory. 

While we won't look at examples of each of the types of memory in LangChain, you can examine the rest here: https://python.langchain.com/en/latest/modules/memory/how_to_guides.html

For reference, some other commonly used memory types are:

- Conversation Knowledge Graph Memory
- ConversationSummaryMemory
- ConversationSummaryBufferMemory
- VectorStore-Backed Memory

## Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. The most common way that indexes are used in chains is in a “retrieval” step. This step refers to taking a user’s query and returning the most relevant documents.

There are 4 main concepts to keep in mind when it comes to working with documents and indexes:
1. Document Loaders
2. Text Splitters
3. VectorStores
4. Retrievers

### Document Loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into “Documents” - a fancy way of say some pieces of text. The document loader is aimed at making this easy. LangChain provides several different document loaders. Lets take a look at a few:

#### PDF Loader

Using PyPDF, we can load any **computer readable** PDF. Note that this is not doing any OCR. 

In [ ]:
from langchain.document_loaders import PyPDFLoader

#first upload a document to drive

!wget "https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/neutralizing_antibodies.pdf" -P papers
!wget "https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/philosophy_of_immunology.pdf" -P papers

--2023-05-23 15:48:10--  https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/antigenic_mapping.pdf
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vanderbilt-data-science/grant-proposal-generation/main/literature/antigenic_mapping.pdf [following]
--2023-05-23 15:48:10--  https://raw.githubusercontent.com/vanderbilt-data-science/grant-proposal-generation/main/literature/antigenic_mapping.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5469040 (5.2M) [application/octet-stream]
Saving to: ‘papers/antigenic_mapping.pdf’

antigenic_mapping.p 100%[=============

In [ ]:
loader = PyPDFLoader("papers/philosophy_of_immunology.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(page_content='PRADEU Philosophy of Immunology Immunology is central to contemporary biology and \nmedicine, but it also provides novel philosophical insights. \nIts most signiﬁ  cant contribution to philosophy concerns the \nunderstanding of biological individuality: what a biological \nindividual is, what makes it unique, how its boundaries are \nestablished and what ensures its identity through time. \nImmunology also oﬀ  ers answers to some of the most \ninteresting philosophical questions. What is the deﬁ  nition of \nlife? How are bodily systems delineated? How do the mind and \nthe body interact? In this Element, Thomas Pradeu considers \nthe ways in which immunology can shed light on these and \nother important philosophical issues. This title is also available \nas Open Access on Cambridge Core at http://dx. doi\n.org/10.1017/9781108616706\nAbout the Series\nThis Cambridge Elements series provides \nconcise and structured introductions to \nall of the central topics in

An advantage of this approach is that documents can be retrieved with page numbers.

We can also load PDF's from a directory:

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("papers/")

docs = loader.load()

#### Loading a HuggingFace dataset

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader

dataset_name="tweet_eval"
page_content_column="text"
name = "stance_climate"

loader=HuggingFaceDatasetLoader(dataset_name,page_content_column, name)

In [ ]:
data = loader.load()

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/355 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/169 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/stance_climate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
data[:5]

[Document(page_content='Why Is The Pope Upset?  via @user #UnzippedTruth #PopeFrancis #SemST', metadata={'label': 0}),
 Document(page_content="We support Australia's Climate Roundtable which is providing a framework for sensible debate ahead of Paris @user #SemST", metadata={'label': 2}),
 Document(page_content="It's nights like this when I'm not so fond of my long hair. I just wanna chop it all off! #heatwave #pnwgirl #SemST", metadata={'label': 0}),
 Document(page_content='#Republican party will go down in history books as party that stood in the way of #gayrights and initiative to reverse #SemST', metadata={'label': 0}),
 Document(page_content='RT @user @user We need degrowth - stop destroying our planet, BP. #SemST', metadata={'label': 2})]

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders([loader])
query = "What are the most used hashtag?"
result = index.query(query)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
result

' The most used hashtags in this context are #SemST, #TakeDownTheFlag, #LoveWins, #Sustainability, and #Environmental.'

### Text Splitters

When you want to deal with long pieces of text, it is necessary to split up that text into chunks. As simple as this sounds, there is a lot of potential complexity here. Ideally, you want to keep the semantically related pieces of text together. What “semantically related” means could depend on the type of text. For example, for simple text, a sentence or paragraph is probably something you want to keep together. 

At a high level, text splitters work as following:
1. Split the text up into small, semantically meaningful chunks (often sentences).
2. Start combining these small chunks into a larger chunk until you reach a certain size (as measured by some function).
3. Once you reach that size, make that chunk its own piece of text and then start creating a new chunk of text with some overlap (to keep context between chunks).

#### Character Splitter

This is the simplest text splitting method, and splits on characters (by default a new-line [\n] character)

In [ ]:
# Download example text file to Colab
!wget https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt

# This is a long document we can split up.
with open('state_of_the_union.txt') as f:
    state_of_the_union = f.read()

--2023-05-23 16:21:38--  https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt.1’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0.004s  

2023-05-23 16:21:38 (9.70 MB/s) - ‘state_of_the_union.txt.1’ saved [39027/39027]



In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.' metadata={}


### Vectorstores

Vectorstores are one of the most important components of building indexes. A key part of working with vectorstores is creating the vector to put in them, which is usually created via embeddings. 

In [ ]:
from langchain.vectorstores import Chroma

with open('state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = Chroma.from_texts(texts, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [ ]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


There are several different types of vectorstores available. We just took a look at the Chromadb vectorstore. Let's take a look at one more open-source vectorstore: **DeepLake**

In [ ]:
from langchain.vectorstores import DeepLake

from langchain.document_loaders import TextLoader

loader = TextLoader('state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

db = DeepLake(dataset_path="./my_deeplake/", embedding_function=embeddings)
db.add_documents(docs)
# or shorter
# db = DeepLake.from_documents(docs, dataset_path="./my_deeplake/", embedding=embeddings, overwrite=True)
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, distance_metric='cos')

### Retrievers

The retriever interface is a generic interface that makes it easy to combine documents with language models.

#### VectorStore Retrievers

Once you construct a VectorStore, its very easy to construct a retriever. Let’s walk through an example.

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('state_of_the_union.txt')

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)


retriever = db.as_retriever()

docs = retriever.get_relevant_documents("what did he say about ketanji brown jackson")

You can also set various parameters to control the behavior of retrievers:


**Maximum Marginal Relevance Retrieval**

By default, the vectorstore retriever uses similarity search. If the underlying vectorstore support maximum marginal relevance search, you can specify that as the search type.

In [ ]:
retriever = db.as_retriever(search_type="mmr")
docs = retriever.get_relevant_documents("what did he say abotu ketanji brown jackson")

**Similarity Score Threshold Retrieval**

You can also a retrieval method that sets a similarity score threshold and only returns documents with a score above that threshold

In [ ]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .5})

docs = retriever.get_relevant_documents("what did he say abotu ketanji brown jackson")

**Specifying top k**

You can also specify search kwargs like k to use when doing retrieval.

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 1})

docs = retriever.get_relevant_documents("what did he say abotu ketanji brown jackson")

len(docs)

#### Other notable Retrievers

**Time Weighted VectorStore**

This retriever uses a combination of semantic similarity and a time decay.
The algorithm for scoring them is:

`semantic_similarity + (1.0 - decay_rate) ** hours_passed`

Notably, hours_passed refers to the hours passed since the object in the retriever was last accessed, not since it was created. This means that frequently accessed objects remain “fresh.”

**Wikipedia Retriever**

WikipediaRetriever has these arguments:
- optional lang: default=”en”. Use it to search in a specific language part of Wikipedia
- optional load_max_docs: default=100. Use it to limit number of downloaded documents. It takes time to download all 100 documents, so use a small number for experiments. There is a hard limit of 300 for now.
- optional load_all_available_meta: default=False. By default only the most important fields downloaded: Published (date when document was published/last updated), title, Summary. If True, other fields also downloaded.

get_relevant_documents() has one argument, query: free text which used to find documents in Wikipedia

In [ ]:
from langchain.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

docs = retriever.get_relevant_documents(query='LangChain')

docs[0].metadata  # meta-information of the Document

In [ ]:
docs[0].page_content[:400]  # content of the Document 

## Chains


Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

### LLMChain

The LLMChain is a simple chain that takes in a prompt template, formats it with the user input and returns the response from an LLM.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))

In [ ]:
# Using multiple variables

prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))

In [ ]:
# Using a ChatModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

You can also chain chains in a sequence using the **SequentialChain**

In [ ]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)

### AnalyzeDocumentChain

The AnalyzeDocumentChain is more of an end to chain. This chain takes in a single document, splits it up, and then runs it through a CombineDocumentsChain. This can be used as more of an end-to-end chain.

**Summarizing**



In [ ]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

from langchain.chains import AnalyzeDocumentChain

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

summarize_document_chain.run(state_of_the_union)

**Question Answering**

In [ ]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(input_document=state_of_the_union, question="what did the president say about justice breyer?")



### Chat Over Documents with Chat History - Putting it all together

**Import all libraries**

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory

**Load Documents**

In [ ]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()

**Split the Documents, Create the Embeddings and Create the VectorStore**

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

**Create a memory object**

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

**Initialize the ConversationalRetrieverChain**

In [ ]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

**Use the Chain**

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query})
result["answer"]

In [ ]:
query = "Did he mention who she suceeded"
result = qa({"question": query})
result['answer']

You can also easily return source documents from the ConversationalRetrievalChain. This is useful for when you want to inspect what documents were returned.

In [ ]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory, return_source_documents=True)

query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query})
result['source_documents'][0]